Setup

In [ ]:
from pathlib import Path
import pandas as pd
import sys

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Load in Data

In [16]:
mar_div_df = pd.read_csv("../data/Marriage_Divorce_rates.csv")
crime_df = pd.read_parquet("../data/summarized_fbi_data.parquet")

Clean up FBI Crime Data

In [21]:
crime_df['month_year'] = pd.to_datetime(crime_df['month_year'])
crime_df['year'] = crime_df['month_year'].dt.year

# Define which columns are counts vs rates
count_cols = [col for col in crime_df.columns if "_actual" in col]
rate_cols = [col for col in crime_df.columns if "_rate" in col or col in ['population','participated_population']]

agg_dict = {col: 'sum' for col in count_cols}
agg_dict.update({col: 'mean' for col in rate_cols})

crime_annual = crime_df.groupby(['state','year']).agg(agg_dict).reset_index()

# Only include years that are in marriage/divorce data
years_to_keep = mar_div_df['year'].unique()
crime_annual = crime_annual[crime_annual['year'].isin(years_to_keep)].reset_index()

crime_annual

,index,state,year,V_actual,V_clearance_actual,ASS_actual,ASS_clearance_actual,BUR_actual,BUR_clearance_actual,LAR_actual,...,HOM_rate,HOM_clearance_rate,RPE_rate,RPE_clearance_rate,ROB_rate,ROB_clearance_rate,ARS_rate,ARS_clearance_rate,P_rate,P_clearance_rate
0,26,Alabama,2011,19666.0,7716.0,13204.0,5611.0,49629.0,6368.0,108787.0,...,0.528333,0.269167,2.367500,0.950000,8.631667,2.535000,1.418333,0.295000,302.635000,56.659167
1,27,Alabama,2012,21148.0,7262.0,14612.0,5488.0,46176.0,5399.0,108195.0,...,0.602500,0.230833,2.260833,0.805000,8.853333,2.129167,1.858333,0.307500,295.379167,52.746667
2,28,Alabama,2013,19725.0,8953.0,13041.0,6263.0,39860.0,5853.0,103176.0,...,0.545833,0.361667,3.605833,1.505000,8.157500,3.084167,1.404167,0.250833,283.402500,62.026667
3,29,Alabama,2014,19603.0,8945.0,12970.0,6209.0,37397.0,5496.0,98385.0,...,0.488333,0.275833,3.419167,1.613333,8.139167,3.077500,2.261667,0.386667,266.225833,61.866667
4,30,Alabama,2015,22117.0,8915.0,15315.0,6331.0,33854.0,4886.0,95735.0,...,0.608333,0.317500,3.490000,1.440833,7.995833,2.838333,1.367500,0.220000,249.524167,55.108333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,2043,Wyoming,2019,1108.0,592.0,734.0,502.0,1229.0,193.0,6555.0,...,0.213333,0.098333,4.860833,0.976667,1.010833,0.393333,0.750000,0.229167,137.991667,27.176667
646,2045,Wyoming,2021,1000.0,444.0,615.0,363.0,1159.0,127.0,5009.0,...,0.260000,0.166667,6.007500,1.046667,0.915000,0.297500,0.987500,0.297500,132.215000,19.350000
647,2046,Wyoming,2022,1001.0,493.0,672.0,422.0,880.0,134.0,5304.0,...,0.258333,0.128333,5.197500,1.050000,0.607500,0.128333,0.662500,0.165000,126.764167,21.361667
648,2047,Wyoming,2023,1055.0,508.0,667.0,414.0,1095.0,193.0,6283.0,...,0.290000,0.160000,4.897500,0.888333,1.081667,0.467500,0.645833,0.126667,131.721667,23.690000


Combine Data

In [22]:
combined_df = pd.merge(
    crime_annual,
    mar_div_df,
    on=['state','year'],
    how='inner'  # only keep states & years present in both
)

combined_df.head()

,index,state,year,V_actual,V_clearance_actual,ASS_actual,ASS_clearance_actual,BUR_actual,BUR_clearance_actual,LAR_actual,...,P_clearance_rate,Unnamed: 0,population_over_15,married_males_last_year,married_females_last_year,divorced_males_last_year,divorced_females_last_year,state.1,married_last_year,divorced_last_year
0,26,Alabama,2011,19666.0,7716.0,13204.0,5611.0,49629.0,6368.0,108787.0,...,56.659167,0,3874758,37987,37530,22071.0,28381.0,1,75517,50452.0
1,27,Alabama,2012,21148.0,7262.0,14612.0,5488.0,46176.0,5399.0,108195.0,...,52.746667,52,3886060,37523,37737,20713.0,22661.0,1,75260,43374.0
2,28,Alabama,2013,19725.0,8953.0,13041.0,6263.0,39860.0,5853.0,103176.0,...,62.026667,132,3910998,36015,34208,23479.0,24495.0,1,70223,47974.0
3,29,Alabama,2014,19603.0,8945.0,12970.0,6209.0,37397.0,5496.0,98385.0,...,61.866667,156,3931966,36382,35000,17639.0,21358.0,1,71382,38997.0
4,30,Alabama,2015,22117.0,8915.0,15315.0,6331.0,33854.0,4886.0,95735.0,...,55.108333,236,3947204,33087,33454,20124.0,19416.0,1,66541,39540.0


Save dataset

In [23]:
combined_df.to_csv("../data/Combined_DF.csv")